# Roughness example

Outline
+ Pull in data
    - Get geopandas dataframe with per-photon length along track, elevation, and filtering flags (including YAPC?)
+ Geophysical corrections
+ Remove solar background photons (optional)
    - From latitude, date, and albedo at 532 nm, compute upwelling shortwave within IS-2 band
    - From field of view and energy per photon (h nu), estimate number of photons per unit time
+ Remove specular returns
+ Correct for bias due to reflectance variability (optional)
+ Deconvolve (either formally or via forward modeling) transmit pulse
    - Try Nathan's forward modeling approach
    - Direct deconvolution (ifft(fft(photon_distribution)/fft(transmit_pulse_distribution)))
+ Deconvolve subsurface reflections
    - Understand methods from Lu et al...

In [1]:
from sliderule import sliderule, earthdata, h5, icesat2

/srv/conda/envs/notebook/lib/python3.10/site-packages/sliderule/sliderule.py:41: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas


In [2]:
sliderule.init(verbose=True)

## Pull in data